# Notebook for Office forecast quarterly Testing

### Set up parameters

In [1]:
# Load packages
import pandas as pd

# Set parameters

curryr = 2022
currqtr = 2
m = 6

prevyr = 2022
prevqtr = 1
prev_m = 3

sector = "Off"

forecast_end_yr = 2031


In [2]:
# Load packages
import pandas as pd
from sklearn import datasets, linear_model, utils, preprocessing
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from datetime import date
from scipy.signal import savgol_filter
from scipy import stats
import seaborn as sns
import re
import dill
import time
import datetime
import seaborn as sns
from pylab import *
from tqdm.notebook import tqdm, trange


# Data File location
genmet_loc = '/home/central/master-data/genmet.dta'
subnames_loc = '/home/central/master-data/subname.dta'
submarkettrend_loc = '/home/central/vc/mfp/finaltoit/data' + str(prevyr) + 'q' + str(prevqtr) + '/data'+ str(prevyr) + 'm' + str(prev_m) + '/qa/submarkettrends.dta'
markettrend_loc = '/home/central/vc/mfp/finaltoit/data' + str(prevyr) + 'q' + str(prevqtr) + '/data'+ str(prevyr) + 'm' + str(prev_m) + '/qa/markettrends.dta'
# apthet_test_loc = '/home/central/metcast/data/' + sector.lower() + '/download/current/apthet-' + str(est_yr) + 'q' + str(est_qtr) + '.log'
ECCAdata_loc = '/home/central/metcast/data/rfa/current/rfa_' + str(curryr) + 'q' + str(currqtr) + '_final.dta'
# submarket_old_loc = '/home/central/vc/mfp/finaltoit/data' + str(est_yr) + 'q' + str(est_qtr) + '/data'+ str(est_yr) + 'm' + str(est_qtr*3) + '/qa/submarkettrends.dta'
offhet_loc = '/home/central/metcast/data/' + sector.lower() + '/download/current/offhet.log'
offsubtrnds_loc = '/home/central/subcast/data/' + sector.lower() + '/download/current/offsubtrnds.log'
ECCA_gdp_loc = '/home/central/cb/ecca/data/output/' + str(curryr) + '_' + str(currqtr*3)+ '/METRO_GDP_PROCESSED.csv'
ECCA_US_loc = '/home/central/cb/ecca/data/output/' + str(curryr) + '_' + str(currqtr*3)+ '/NATIONAL_SET_PROCESSED.csv'
offmets_loc = '/home/central/master-do-files/keep_US_offmets.do'
df_rol_loc = '/home/central/subcast/data/' + sector.lower() + '/forecast/current/offsubtest_'+ str(prevyr) + 'q' + str(prevqtr) +'.dta'


# Load Data
genmet = pd.read_stata(genmet_loc)
offmets = open(offmets_loc, "r").read()
offmets_list = re.findall(r'"(.*?)"', offmets)

submarkettrend = pd.read_stata(submarkettrend_loc)
markettrend = pd.read_stata(markettrend_loc)
df_offhet = pd.read_table(offhet_loc, header =0)
df_offhet = df_offhet.dropna(axis=1, how='all')
df_offhet.columns = df_offhet.columns.str.replace(' ', '')
df_offhet.loc[df_offhet.MSA.isna(), "MSA"] = "NA"
df_offhet = df_offhet.drop(0)
df_offhet.Geosub = df_offhet.Geosub.astype('int')
df_offhet.Year = df_offhet.Year.astype('int')
df_offhet.Qtr = df_offhet.Qtr.astype('int')
df_offhet.Size = df_offhet.Size.astype('int')

df_offsubtrnds = pd.read_csv(offsubtrnds_loc)
df_offsubtrnds.loc[df_offsubtrnds.msa.isna(), "msa"] = "NA"

df_gdp = pd.read_csv(ECCA_gdp_loc)
df_ecca_US = pd.read_csv(ECCA_US_loc)
df_gdp.loc[df_gdp.metcode.isna(), "metcode"] = "NA"
df_gdp_dc = df_gdp[df_gdp.metcode=="SV"]
df_gdp_dc.metcode = "DC"
df_gdp = pd.concat([df_gdp,  df_gdp_dc])


df_rfa = pd.read_stata(ECCAdata_loc)

# submarket_old = pd.read_stata(submarket_old_loc)
df_rol = pd.read_stata(df_rol_loc)
df_rol = df_rol[["metcode", "subid", "yr", "qtr", "cons", "h","e", "Bcon", "abs", "Babs",
                 "vac", "Bvac", "mrent", "Bmrent" ,"G_mrent" ,"merent", "G_merent", "Bmerent", "gap", "Bgap"]]
df_rol = df_rol.rename(columns = { 'h' : 'rol_h', 'e' : 'rol_e', 'cons' : 'rolscon',
                                'Bcon' : 'rolcons', 'abs' : 'rolsabs', 'vac' : 'rolsvac',
                                'Babs' : 'Brolabs', 'mrent' : 'rolsmre', 'Bvac' : 'rolvac',
                                'Bmrent' : 'rolmrent', 'G_mrent' : 'grolsmre',
                                'merent' : 'rolmere', 'G_merent' : 'grolsmer', 
                                'Bmerent' : 'Brolmeren', 'gap' : 'rolsgap', 
                                'Bgap' : 'Brolgap'
                         })


# output_df = pd.read_stata('/home/central/subcast/data/off/forecast/current/offsubdiag_2020q4_OOB.dta')

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-j0jalmu8 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [9]:
df_output_template = pd.read_stata('/home/central/subcast/data/off/forecast/current/offsubdiag_2020q4_OOB.dta')
df_output_template.columns

Index(['metcode', 'subid', 'yr', 'qtr', 'inv', 'cons', 'rolscon', 'h', 'rol_h',
       'e', 't', 'Bcon', 'rolcons', 'demo', 'conv', 'occ', 'abs', 'rolsabs',
       'Babs', 'Brolabs', 'vac', 'rolsvac', 'Bvac', 'rolvac', 'mrent',
       'rolsmre', 'Bmrent', 'rolmrent', 'G_mrent', 'grolsmre', 'G_Bmrent',
       'merent', 'G_merent', 'grolsmer', 'Bmerent', 'Brolmeren', 'G_Bmerent',
       'gap', 'rolsgap', 'Bgap', 'Brolgap', 'cons_oob', 'vac_oob', 'abs_oob',
       'G_mrent_oob', 'G_merent_oob', 'gap_oob'],
      dtype='object')

In [14]:
sub_list = submarkettrend[(submarkettrend.sector==sector)][['metrocode', "subid"]].drop_duplicates()
input_list = df_offsubtrnds[['msa','subid']].drop_duplicates()

print("There are", sub_list.shape[0], "submarkets in the submarket trend and ", input_list.shape[0], "submarkets in the aptsubtrnds in " , sector, "sector.")
input_list = input_list[input_list.subid!=90]
print("There are", input_list.shape[0], "submarkets in the aptsubtrnds after excluding subid 90")
df_list = pd.merge(sub_list, input_list, left_on=['metrocode', "subid"], right_on = ['msa','subid'], how='right', indicator='Exist')
df_list.Exist.unique()

print('Office trend and submarket covers the same number of submarkets. No submarkets are surpressed.')

There are 846 submarkets in the submarket trend and  846 submarkets in the aptsubtrnds in  Off sector.
There are 846 submarkets in the aptsubtrnds after excluding subid 90
Office trend and submarket covers the same number of submarkets. No submarkets are surpressed.


In [21]:
submarkettrend[submarkettrend.sector=="Off"][['sector', 'subsector', 'metrocode', 'subid', 'year', 'quarter', 'vacpercent', 'vacpercentptchg']]
df_submarket_yr = submarkettrend[(submarkettrend.sector==sector) & (submarkettrend.quarter!=5 )][['metrocode', 'subid', 'year']].groupby(['metrocode', 'subid']).min().reset_index()
print('Number of submarket by starting year.')
print(df_submarket_yr.groupby("year").count()) 
df_metro_yr = markettrend[(markettrend.sector==sector ) & (markettrend.quarter!=5 )][['metrocode', 'year']].groupby(['metrocode']).min().reset_index()
print('Number of metros and regions by starting year.')
print(df_metro_yr.groupby("year").count()) 

df_submarket_yr = df_submarket_yr.rename(columns={"year": "submarket_yr_begin"})
df_metro_yr = df_metro_yr.rename(columns={"year": "metro_yr_begin"})
df_submarket_yr = df_submarket_yr.merge(df_metro_yr, on = ['metrocode'], how='outer')
df_submarket_yr

Number of submarket by starting year.
        metrocode  subid
year                    
1999.0        445    445
2000.0        164    164
2001.0         12     12
2002.0          1      1
2005.0          9      9
2007.0         98     98
2011.0        113    113
2019.0          4      4
Number of metros and regions by starting year.
        metrocode
year             
1999.0         56
2000.0         42
2005.0          2
2007.0         50
2008.0          6
2011.0         58


,metrocode,subid,submarket_yr_begin,metro_yr_begin
0,AA,81.0,2011.0,2011.0
1,AA,82.0,2011.0,2011.0
2,AB,81.0,2007.0,2007.0
3,AB,82.0,2007.0,2007.0
4,AC,81.0,2011.0,2011.0
...,...,...,...,...
865,R2,NaN,NaN,2000.0
866,R3,NaN,NaN,2000.0
867,R4,NaN,NaN,2000.0
868,R5,NaN,NaN,2000.0


In [13]:
# tertiary_market = (metro.isin(['BB', 'BD','BK','FM','FP','FR','FS','LL','MB','MD','NP','RE','SK','SR','SS','ST','TL','VJ','PR','HL','BY','QN']) | (subid.isin([81,82]))



True

In [16]:
metro = "AT"
subid = 1
# Structure H_E stock data
df_HE = pd.DataFrame(df_offhet[(df_offhet.MSA==metro) & (df_offhet.Geosub==subid) & (df_offhet.H_E=="H")].groupby(['Year','Qtr']).Size.sum().reset_index())
df_HE = df_HE.rename(columns={"Size": "H"})
temp = pd.DataFrame(df_offhet[(df_offhet.MSA==metro) & (df_offhet.Geosub==subid) & (df_offhet.H_E=="E")].groupby(['Year','Qtr']).Size.sum().reset_index())
temp = temp.rename(columns={"Size": "E"})
df_HE = df_HE.merge(temp, on = ['Year','Qtr'], how='outer')
df_HE = df_HE.sort_values(['Year', 'Qtr'])

df = df_offsubtrnds[(df_offsubtrnds.msa==metro) & (df_offsubtrnds.subid==subid)]
df = df.rename(columns = { 'msa' : 'metrocode', 'construction' : 'completions', 
                            'qtr' : 'quarter', 'absorption' : 'netabsorption', 'vacancy' : 'vacpercent', 
                            'avgrent' : 'marketrentdollars', 
                            'occ_stock' : 'occupiedstock', 'effrent' : 'effrentdollars'})
df_qtr5 = df[(df.quarter==5)]
df = df[(df.quarter!=5)].dropna(how='all', axis = 1)  # select quarterly data
df['marketrentpctchange'] = df.marketrentdollars.pct_change(fill_method=None)
df['effrentpchange'] = df.effrentdollars.pct_change(fill_method=None)
df['vacpercentptchg'] = df.vacpercent.diff()
df

,msa,metro,sector,subid,subname,year,qtr,inventory,construction,vacancy,avgrent,effrent,demolitions,occ_stock,conversions,absorption
1944,AT,Atlanta,Office,1,Cumberland/I-75,1989,5,16364000.0,NaN,0.223,19.400000,15.07,NaN,12715000.0,NaN,NaN
1945,AT,Atlanta,Office,1,Cumberland/I-75,1990,5,16719000.0,355000.0,0.214,18.559999,14.37,0.0,13141000.0,0.0,426000.0
1946,AT,Atlanta,Office,1,Cumberland/I-75,1991,5,17536000.0,817000.0,0.187,17.879999,14.55,0.0,14257000.0,0.0,1116000.0
1947,AT,Atlanta,Office,1,Cumberland/I-75,1992,5,17636000.0,100000.0,0.159,17.340000,14.40,0.0,14832000.0,0.0,575000.0
1948,AT,Atlanta,Office,1,Cumberland/I-75,1993,5,17627000.0,236000.0,0.128,17.010000,14.88,0.0,15371000.0,-245000.0,539000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,AT,Atlanta,Office,1,Cumberland/I-75,2021,3,21088000.0,336000.0,0.200,27.510000,21.71,0.0,16871000.0,0.0,195000.0
2067,AT,Atlanta,Office,1,Cumberland/I-75,2021,4,21088000.0,0.0,0.196,27.510000,21.81,0.0,16948000.0,0.0,77000.0
2068,AT,Atlanta,Office,1,Cumberland/I-75,2021,5,21088000.0,336000.0,0.196,27.510000,21.81,0.0,16948000.0,-96000.0,-89000.0
2069,AT,Atlanta,Office,1,Cumberland/I-75,2022,1,21088000.0,0.0,0.204,27.570000,21.48,0.0,16778000.0,0.0,-170000.0


In [79]:
## Master Program
# Forecast with existing input-output data format
# Take estimated models in pkl files; new submarkets use metro pkl models 

start_time = time.time()
print("Forecast starts at ",time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time)))

try:
    del df_output
    del df_output_y
except:
    pass

# Selecct a market for prototype
# Monthly data from 2009-2019, total 132 months
# Quarterly data from 1999 - 2019, total 84 quarters
# Annual data from 1990- 2019, total 30 years
sub_list = submarkettrend[(submarkettrend.sector==sector)][['metrocode', "subid"]].drop_duplicates()
# count_metro = markettrend[(markettrend.sector==sector)][['metrocode']].drop_duplicates().shape[0]
input_list = df_offsubtrnds[['msa','subid']].drop_duplicates()

print("There are", sub_list.shape[0], "submarkets in the submarket trend and ", input_list.shape[0], "submarkets in the aptsubtrnds in " , sector, "sector.")
input_list = input_list[input_list.subid!=90]
print("There are", input_list.shape[0], "submarkets in the aptsubtrnds after excluding subid 90")
df_list = pd.merge(sub_list, input_list, left_on=['metrocode', "subid"], right_on = ['msa','subid'], how='right', indicator='Exist')
df_submarket_yr = submarkettrend[(submarkettrend.sector==sector) & (submarkettrend.quarter!=5 )][['metrocode', 'subid', 'year']].groupby(['metrocode', 'subid']).min().reset_index()
df_submarket_yr = df_submarket_yr.rename(columns={"year": "submarket_yr_begin"})

# # subset testing
# metro = "OA"
# subid = 1
# sub_list = sub_list[(sub_list.metrocode==metro) & (sub_list.subid==subid)]

for row in trange(len(df_list.to_dict("split")["data"]), desc='Office Submarket Forecast Progress'):
# for row in range(len(df_list.to_dict("split")["data"])):
# for row in [0]:
    metro = df_list.to_dict("split")["data"][row][2]
    subid = df_list.to_dict("split")["data"][row][1]
#     print("forecasting ", metro, subid)
# for row in range(len(sub_list.to_dict("split")["data"])):
#     metro = sub_list.to_dict("split")["data"][row][0]
#     subid = sub_list.to_dict("split")["data"][row][1]
#     metro = "CR"
#     subid = 12

    tertiary_market = (metro in ['BB', 'BD','BK','FM','FP','FR','FS','LL','MB','MD','NP','RE','SK','SR','SS','ST','TL','VJ','PR','HL','BY','QN']) | (subid in [81,82])
    
    # Structure H_E stock data
    df_HE = pd.DataFrame(df_offhet[(df_offhet.MSA==metro) & (df_offhet.Geosub==subid) & (df_offhet.H_E=="H")].groupby(['Year','Qtr']).Size.sum().reset_index())
    df_HE = df_HE.rename(columns={"Size": "H"})
    temp = pd.DataFrame(df_offhet[(df_offhet.MSA==metro) & (df_offhet.Geosub==subid) & (df_offhet.H_E=="E")].groupby(['Year','Qtr']).Size.sum().reset_index())
    temp = temp.rename(columns={"Size": "E"})
    df_HE = df_HE.merge(temp, on = ['Year','Qtr'], how='outer')
    df_HE = df_HE.sort_values(['Year', 'Qtr'])

    df = df_offsubtrnds[(df_offsubtrnds.msa==metro) & (df_offsubtrnds.subid==subid)]
    df = df.rename(columns = { 'msa' : 'metrocode', 'construction' : 'completions', 
                                'qtr' : 'quarter', 'absorption' : 'netabsorption', 'vacancy' : 'vacpercent', 
                                'avgrent' : 'marketrentdollars', 
                                'occ_stock' : 'occupiedstock', 'effrent' : 'effrentdollars'})
    df_qtr5 = df[(df.quarter==5)]
    df = df[(df.quarter!=5)].dropna(how='all', axis = 1)  # select quarterly data
    df['marketrentpctchange'] = df.marketrentdollars.pct_change(fill_method=None)
    if not tertiary_market:
        df['effrentpchange'] = df.effrentdollars.pct_change(fill_method=None)
    else:
        pass
    df['vacpercentptchg'] = df.vacpercent.diff()    
    
    df_ecca = df_rfa.merge(df_gdp, on = ['metcode', 'yr', 'qtr'],how = 'left')
    df_ecca = df_ecca.merge(df_ecca_US, left_on = ['yr', 'qtr'], right_on = ['YR', 'QTR'],how = 'left')
    df = df.merge(df_ecca[(df_ecca.metcode==metro) & (df_rfa.qtr!=5)], left_on=['metrocode', 'year', 'quarter'], 
                  right_on=['metcode', 'yr', 'qtr'], how='outer' )
    df = df.merge(df_HE, left_on=['yr', 'qtr'], right_on=['Year', 'Qtr'], how = 'left')

    df.loc[(df.H.isna()) & (df.yr < curryr+3), 'H'] = 0
    df['t'] = df.E + df.H
        
    yr_earlist = df[df.metrocode.notna()].year.min()
    df = df[(df.yr >= yr_earlist ) & (df.yr<= curryr + 10)]
    df['Month'] = df.qtr*3
    df['time'] = pd.to_datetime(df['yr'].astype(int).astype(str)  + df['Month'].astype(int).astype(str), format='%Y%m') 
    df['rn'] = np.random.uniform(0.5,1.5, df.shape[0])

    df= df.sort_values(['yr', 'qtr'])
    df = df[df.yr <= forecast_end_yr]

    df_copy = df.copy()

    ## Transform all level data to percentage change
    df['inv_chg'] = df.inventory.pct_change(fill_method=None).shift()
    df['inv_lag'] = df['inventory'].shift()
    df['completions_lag'] = df['completions'].shift()
    df['qfet_chg'] = df.qfet.pct_change(fill_method=None)

    ### calculate z-scores
    cols = ['gdp',  'qfet', 'offemp', 'YIELD12M', 'CREPIOFF', 'qfet_chg']
    for col in cols:
        col_zscore = col + '_zscore'
        df[col_zscore] = (df[col] - df[col].mean())/df[col].std(ddof=0)
    df = df.replace([np.inf, -np.inf], 0)


    ## Set up threshold based on survey construction stocks
    df.loc[(df.E.isnull()) & (df.time>= str(curryr-3) + "-" + str(currqtr*3)),'E'] = 0
    df.loc[df.H.isnull() & (df.time>= str(curryr-1) + "-" + str(currqtr*3)),'H'] = 0
    df['temp'] = df.E * 0.2 + df.H
    df.loc[df.temp.isna(), 'temp'] = df[df.temp.isna()].completions
    df['threshold'] = df.temp.rolling(6).mean()
    df.loc[df.threshold.isna(), 'threshold'] = df.completions.rolling(6).mean()
    condition = (df.qfet_chg_zscore >= 1.) & (df.qfet_chg_zscore < 1.5)
    df.loc[condition, 'threshold'] = df[condition].threshold * 0.6
    condition = (df.qfet_chg_zscore > 0.) & (df.qfet_chg_zscore < 1.)
    df.loc[condition, 'threshold'] = df[condition].threshold * 0.5
    condition = (df.qfet_chg_zscore <= 0.) 
    df.loc[condition, 'threshold'] = df[condition].threshold * 0.4
    condition = (df.time>= str(curryr+3) + "-" + str(currqtr*3))
    df.loc[condition, 'threshold'] = df[(df.time>= str(curryr-15) + "-" + str(currqtr*3))].completions.mean() * df[condition].qfet_chg_zscore *0.4
    condition = (df.time>= str(curryr+3) + "-" + str(currqtr*3)) & (df.qfet_chg_zscore  < 0.)
    gfet_max = df[condition].qfet_chg_zscore.max()
    gfet_min = df[condition].qfet_chg_zscore.min()
    df.loc[condition, 'threshold'] = df[(df.time>= str(curryr-15) + "-" + str(currqtr*3))].completions.mean() *(0.97+ 0.06* (df[condition].qfet_chg_zscore - gfet_min) / (gfet_max - gfet_min))   
    condition = (df.time>= str(curryr+2) + "-" + str(currqtr*3)) & (df.threshold < df[df.completions>10000].completions.min())
    df.loc[condition, 'threshold'] = df[df.completions >= 10000].completions.min() * df[condition].rn
    
    condition = (df.H > 0) & (df.threshold < df.H )
    df.loc[condition, 'threshold'] = df[condition].H
    con =  (df.completions.isna()) & (df.time>= str(curryr) + "-" + str(currqtr*3)) & (df.time< str(curryr+2) + "-" + str(currqtr*3))
    df.loc[con, 'threshold'] = df[con].H + df[con]['E']/4
    condition = (df.threshold.isnull()) 
#     df.loc[condition, 'threshold'] = df[(df.time>= str(curryr-15) + "-" + str(currqtr*3))].completions.mean()
#     df.loc[df.threshold < 10000, "threshold"] = df[df.completions >= 10000].completions.min()* df[df.threshold < 10000].rn
    df.loc[df.threshold.isnull(), "threshold"] = 0

    if not tertiary_market:
        ## Select Variables
        X_REIS = ['inv_lag','completions_lag']
        X_ECCA = ['gdp_zscore',  'qfet_zscore', 'YIELD12M_zscore', 'CREPIOFF_zscore']
        X_select = X_REIS + X_ECCA
        df[X_ECCA] = df[X_ECCA].fillna(method='ffill')

        X_list = X_select
        if float(df_submarket_yr[(df_submarket_yr.metrocode==metro) & (df_submarket_yr.subid== subid)].submarket_yr_begin) <= 2011:
            clf = dill.load( open('./Model_off/con_' + metro+'_'+str(int(subid)) +'.pkl', "rb" ) )
        else:
            clf = dill.load( open('./Model_off/metro_model/con_' + metro+ '.pkl', "rb" ) )
    #         clf.intercept_ = df.completions.mean()
            adjustment_factor = df[df.completions.notnull()].completions.mean() / clf.predict(df[(df.completions.notna()) & (df.completions_lag.notnull()) & (df.yr >2015) & (df.inv_lag.notna())][X_list]).mean()

    #     # Use H stock to fill the coming 8 quarters
        con =  (df.completions.isna()) & (df.time>= str(curryr) + "-" + str(currqtr*3)) & (df.time< str(curryr+2) + "-" + str(currqtr*3))
        # around 25% E stock would be completed in the planned year
        df['conE1'] = df.E.rolling(8).mean() *.10  
        # around 10% E stock would be completed in the planned year
        df['conE2'] = df.E.rolling(8).mean() *.05
        df['conE2'] = df['conE2'].shift(4)
        df.loc[con, 'completions'] = df[con]['H'] + df[con]['conE1'] + df[con]['conE2']
        df.loc[df.completions < 10000, "completions"] = 0.
        df['completions'] = round(df['completions'])
        df['completions_lag'] = df['completions'].shift()
        
        n=1
        while n <=8:
            df.loc[(df.time>= str(curryr) + "-" + str(currqtr*3)), 'inventory'] = df[(df.time>= str(curryr) + "-" + str(currqtr*3))].inv_lag + df[(df.time>= str(curryr) + "-" + str(currqtr*3))].completions
            df['inv_lag'] = df['inventory'].shift() 
            n = n+1
        df['inv_chg'] = df.inventory.pct_change(fill_method=None).shift()

        try:
            while df[df.yr >curryr-1].completions.isna().sum()>0:
                if float(df_submarket_yr[(df_submarket_yr.metrocode==metro) & (df_submarket_yr.subid== subid)].submarket_yr_begin) <= 2011:
                    df.loc[(df.completions.isna()) & (df.yr >2015) & (df.inv_lag.notna()), 'completions'] =clf.predict(df[(df.completions.isna()) & (df.yr >2015) & (df.inv_lag.notna())][X_list])
                else:
                    df.loc[(df.completions.isna()) & (df.yr >2015) & (df.inv_lag.notna()), 'completions'] =clf.predict(df[(df.completions.isna()) & (df.yr >2015) & (df.inv_lag.notna())][X_list]) * adjustment_factor
                con = ((df.time> str(curryr+2) + "-" + str(currqtr*3)) & (df.completions < 10000) & (df.completions > 0))
                df.loc[con, "completions"] = 10000 * df[con].rn
                df.loc[(df.time> str(curryr) + "-" + str(currqtr*3)) & (df.completions < 10000), "completions"] = 0
                df.loc[(df.completions > df.threshold)  & (df.time>= str(curryr) + "-" + str(currqtr*3)), "completions"] = df[(df.completions > df.threshold)  & (df.time>= str(curryr) + "-" + str(currqtr*3))].threshold
                df['completions'] = round(df['completions'])
                df.loc[(df.inventory.isna()) & (df.inv_lag.notna()), 'inventory'] = df[(df.inventory.isna()) & (df.inv_lag.notna())].inv_lag + df[(df.inventory.isna()) & (df.inv_lag.notna())].completions
                df['inv_lag'] = df['inventory'].shift()   
                df['inv_chg'] = df.inventory.pct_change(fill_method=None).shift()
                df['completions_lag'] = df['completions'].shift()
        except:
            print(metro, subid,"Something in the construction forecast went wrong")



        # Forecast Vacancy Model (Hendershott 2009)
        ## Prepare data
        df['qfet_ln_diff'] = np.log(df['qfet']).diff()
        df['qfet_ln_diff_lag'] = np.log(df['qfet']).diff().shift()
        df['qfet_ln_diff_lag2'] = np.log(df['qfet']).diff().shift(2)
        df['USGDP_ln_diff'] = df['USGDP'].pct_change(periods=4)
        df['USGDP_ln_diff_lag'] = df['USGDP'].pct_change(periods=4).shift()
        df['gdp_ln_diff'] = df['gdp'].pct_change(periods=4)
        df['gdp_ln_diff_lag'] = df['gdp'].pct_change(periods=4).shift()
        df['inv_ln_diff'] = np.log(df['inventory']).diff()
        df['vac_lag'] = (df.vacpercent).shift()
        df['vacpercentptchg_lag'] = (df.vacpercentptchg).shift()

        ## Select Variables
        X_REIS = ['inv_ln_diff', 'vac_lag','vacpercentptchg_lag']
        X_ECCA = ['qfet_ln_diff', 'qfet_ln_diff_lag', 'qfet_ln_diff_lag2', 'USGDP_ln_diff', 'USGDP_ln_diff_lag']
        X_select = X_REIS + X_ECCA
        df[X_ECCA] = df[X_ECCA].fillna(method='ffill')

        con = (df.time> str(curryr-10) + "-" + str(currqtr*3)) & (df.qtr == 4)
        yd_min = np.min(df[con].vacpercentptchg)
        yd_max = np.max(df[con].vacpercentptchg)
        yd_std = np.std(df[con].vacpercentptchg)
        y_min = np.min(df[con].vacpercent)
        y_max = np.max(df[con].vacpercent)
        y_std = np.std(df[con].vacpercent)

        y_last = list(df[["vacpercent"]].dropna().vacpercent)[-1]
        y_mean = np.mean(df[df.time> str(curryr-10) + "-" + str(currqtr*3)].vacpercent)
        if y_std > 0.05:
            y_mean = np.nanpercentile(df[df.time> str(curryr-10) + "-" + str(currqtr*3)].vacpercent, 50)
            y_min = np.nanpercentile(df[df.time> str(curryr-10) + "-" + str(currqtr*3)].vacpercent, 10)  

        X_list = X_select
        if float(df_submarket_yr[(df_submarket_yr.metrocode==metro) & (df_submarket_yr.subid== subid)].submarket_yr_begin) <= 2011:
            clf = dill.load( open('./Model_off/vac_' + metro+'_'+str(int(subid)) +'.pkl', "rb" ) )
        else:
            clf = dill.load( open('./Model_off/metro_model/vac_' + metro+ '.pkl', "rb" ) )
#             adjustment_factor = df.vacpercentptchg.mean() / clf.predict(df[(df.vacpercentptchg.notna()) & (df.yr >2015) & (df.vacpercentptchg.notna()) & (df.vacpercentptchg_lag.notnull())][X_list]).mean()

        try:
            while df[df.yr >curryr-1].vacpercentptchg.isna().sum()>0:
                try:
                    if delay_stress > 0:
                        df.loc[(df.vacpercentptchg.isna()) & (df.yr >2015) & (df.vac_lag.notna()), 'vacpercentptchg'] = (clf.predict(df[(df.vacpercentptchg.isna()) & (df.yr >2015) & (df.vac_lag.notna())][X_list])) + delay_stress
                    else:
                        df.loc[(df.vacpercentptchg.isna()) & (df.yr >2015) & (df.vac_lag.notna()), 'vacpercentptchg'] = (clf.predict(df[(df.vacpercentptchg.isna()) & (df.yr >2015) & (df.vac_lag.notna())][X_list])) 
                except:
                    df.loc[(df.vacpercentptchg.isna()) & (df.yr >2015) & (df.vac_lag.notna()), 'vacpercentptchg'] = (clf.predict(df[(df.vacpercentptchg.isna()) & (df.yr >2015) & (df.vac_lag.notna())][X_list])) 


                QA_condition = (df.vacpercentptchg + df.vac_lag <= y_min)
                df.loc[QA_condition, "vacpercentptchg"] = y_min - df[QA_condition].vac_lag

                QA_condition = (df.vacpercentptchg < yd_min)
                df.loc[QA_condition, "vacpercentptchg"] = yd_min
                QA_condition = (df.vacpercentptchg > yd_max )
                delay_stress = list(df[df.vac_lag.notna()].vacpercentptchg)[-1] - (yd_max )
                df.loc[QA_condition, "vacpercentptchg"] = yd_max 
                con = (df.time> str(curryr+5) + "-" + str(currqtr*3)) & (df.vacpercentptchg > -0.0001)
                df.loc[con, "vacpercentptchg"] = -0.0001
                df.loc[(df.vacpercent.isna()) & (df.vac_lag.notna()), 'vacpercent'] = df[(df.vacpercent.isna()) & (df.vac_lag.notna())].vac_lag  + df[(df.vacpercent.isna()) & (df.vac_lag.notna())].vacpercentptchg
                ## Adding movements
                con = (df.time> str(curryr+2) + "-" + str(currqtr*3)) & (df.vacpercent.round(decimals= 2).diff()==0) & (df.vacpercent.round(decimals= 2).diff(periods=2)==0)
                df.loc[con, "vacpercent"] = df[con]["vacpercent"] + (df[con]["rn"]-1)/5 * df[con]["vacpercentptchg"]
                con = (df.time> str(curryr+2) + "-" + str(currqtr*3)) & (df.vacpercent.round(decimals= 2).diff()==0) & (df.vacpercent.round(decimals= 2).diff(periods=2)==0)
                df.loc[con, "vacpercent"] = df[con]["vacpercent"] + (df[con]["rn"]-1)/30 * y_std
                con = (df.time> str(curryr) + "-" + str(currqtr*3)) & (df.vacpercent < y_min)
                df.loc[con, "vacpercent"] = y_min + (df[con]["rn"]-1) * yd_std
                con = (df.time> str(curryr) + "-" + str(currqtr*3)) & (df.vacpercent < y_min)
                df.loc[con, "vacpercent"] = y_min
                df['vac_lag'] = (df.vacpercent).shift()
                df['vacpercentptchg_lag'] = (df.vacpercentptchg).shift()

        except:
            print(metro, subid, "Something in the vacancy forecast went wrong")

    #     # Limit the vacpercent long term forecast to long term mean after 5th year
    #     LR_start = 0
    #     df['ct'] = 1
    #     df.loc[df.time<= str(curryr+ LR_start) + "-" + str(currqtr*3), 'ct'] = 0
    #     df.loc[df.time >  str(curryr+ LR_start + 5) + "-" + str(currqtr*3), 'ct'] = 0
    #     ct_count = df.ct.sum()
    #     df['ct'] = df['ct'].cumsum() / ct_count
    #     df.loc[df.time >  str(curryr+ LR_start + 5) + "-" + str(currqtr*3), 'ct'] = 1

    #     y_f_last = list(df.vacpercent)[-1]
    #     y_last5 = float(df[df.time== str(curryr+ LR_start) + "-" + str(currqtr*3)].vacpercent)

    #     if ((y_f_last > y_last5) & (y_last5 > y_mean)) :
    #         y_last5 = float(df[df.time== str(curryr+ LR_start + 4) + "-" + str(currqtr*3)].vacpercent)
    #         ratio_adj = ((y_mean )-y_last5)/(y_f_last- y_last5)
    #         con = (df.time> str(curryr+ LR_start + 4) + "-" + str(currqtr*3))
    #         df.loc[con, "vacpercent"] =float(y_last5) + float(ratio_adj) * (df[con].vacpercent - float(y_last5))

    #     elif (y_f_last > y_mean) & (y_mean > y_last5):
    #         y_last5 = float(df[df.time== str(curryr+ LR_start + 4) + "-" + str(currqtr*3)].vacpercent)
    #         ratio_adj = ((y_mean )-y_last5)/(y_f_last- y_last5)
    #         con = (df.time> str(curryr+ LR_start + 4) + "-" + str(currqtr*3))
    #         df.loc[con, "vacpercent"] =float(y_last5) + float(ratio_adj) * (df[con].vacpercent - float(y_last5))

    #     elif (y_f_last != y_mean) & (y_mean == y_last5):
    #         y_last5 = float(df[df.time== str(curryr+ LR_start + 4) + "-" + str(currqtr*3)].vacpercent)
    #         ratio_adj = ((y_mean )-y_last5)/(y_f_last- y_last5)
    #         con = (df.time> str(curryr+ LR_start + 4) + "-" + str(currqtr*3))
    #         df.loc[con, "vacpercent"] =float(y_last5) + float(ratio_adj) * (df[con].vacpercent - float(y_last5))

    #     elif (y_f_last < y_mean) & (y_mean < y_last5):
    #         if  y_mean - y_f_last > 0.5 * y_std:
    #             ratio_adj = ((y_mean - 0.5 *y_std )-y_last5)/(y_f_last - y_last5)
    #             con = (df.time> str(curryr+ LR_start) + "-" + str(currqtr*3))
    #             df.loc[con, "vacpercent"] =float(y_last5) + float(ratio_adj) * (df[con].vacpercent - float(y_last5))


    #     elif ((y_last5 > y_f_last) & (y_f_last > y_mean)) | ((y_last5 < y_f_last) & (y_f_last < y_mean)):
    #         if  abs(y_mean - y_f_last) > 0.5 * y_std:
    #             con = (df.time> str(curryr+ LR_start) + "-" + str(currqtr*3))
    #             ratio_adj =  np.sign(y_mean - y_f_last) * (abs(y_mean - y_f_last) - 0.5 * y_std)
    #             df.loc[con, "vacpercent"] = df[con].vacpercent + (float(ratio_adj)) * df.ct

    #     elif ((y_f_last < y_last5) & (y_last5 < y_mean)):
    #         con = (df.time> str(curryr+ LR_start) + "-" + str(currqtr*3)) 
    #         ratio_adj =  y_mean - y_f_last
    #         if ratio_adj < 2*y_std:
    #             df.loc[con, "vacpercent"] = df[con].vacpercent + (float(ratio_adj)) * df.ct
    #         else:
    #             df['ct'] = 1
    #             df.loc[df.time<= str(curryr+ LR_start) + "-" + str(currqtr*3), 'ct'] = 0
    #             df.loc[df.time >  str(curryr+ LR_start + 4) + "-" + str(currqtr*3), 'ct'] = 0
    #             ct_count = df.ct.sum()
    #             df['ct'] = df['ct'].cumsum() / ct_count
    #             df.loc[df.time >  str(curryr+ LR_start + 4) + "-" + str(currqtr*3), 'ct'] = 1
    #             df.loc[con, "vacpercent"] = df[con].vacpercent + (float(ratio_adj)) * df.ct


    #     # Limit the vacancy max  based on history
    #     y_hat_max = np.max(df[df.time> str(curryr) + "-" + str(currqtr*3)].vacpercent)
    #     upper_bd = y_max 
    #     if y_hat_max > upper_bd  :
    #         ratio_adj = ((upper_bd )-y_last)/(y_hat_max- y_last)
    #         con = (df.time> str(curryr) + "-" + str(currqtr*3)) & (df.vacpercent > y_last)
    #         if ratio_adj != 0:
    #             df.loc[con, "vacpercent"] = float(y_last) + float(ratio_adj) * (df[con].vacpercent - float(y_last))
    #         else:
    #             df.loc[con, "vacpercent"] = upper_bd *(1 + (df[con]["rn"]-1)/10 * (yd_std))
    #             df.loc[(df.time> str(curryr) + "-" + str(currqtr*3)) & (df.vacpercent > upper_bd), "vacpercent"] = upper_bd


    #     # Limit the vacancy min  based on history
    #     y_hat_min = np.min(df[df.time> str(curryr) + "-" + str(currqtr*3)].vacpercent)
    #     lower_bd = y_min
    #     if y_hat_min < lower_bd :
    #         ratio_adj = ((lower_bd)-y_last)/(y_hat_min- y_last)
    #         con = (df.time> str(curryr) + "-" + str(currqtr*3)) & (df.vacpercent < y_last)
    #         if ratio_adj != 0:
    #             df.loc[con, "vacpercent"] = float(y_last) + float(ratio_adj) * (df[con].vacpercent - float(y_last))
    #         else:
    #             df.loc[con, "vacpercent"] = y_min *(1 + (df[con]["rn"]-1)/10 * (yd_std))
    #             df.loc[(df.time> str(curryr) + "-" + str(currqtr*3)) & (df.vacpercent < lower_bd), "vacpercent"] = lower_bd

    #     # Smooth if there is unusual volatility in the long term
    #     con = (df.time> str(curryr+ 2) + "-" + str(currqtr*3)) & (abs(df.vacpercent.pct_change(fill_method=None)) > yd_std)
    #     df.loc[con, 'vacpercent'] = np.nan
    #     df['vacpercent'] = df['vacpercent'].interpolate()

    #     # Recalculate vacpercentptchg
    #     df['vacpercentptchg'] = df.vacpercent.diff()

    #     # Smooth if there is vacpercentptchg above historical min/max
    #     con = (df.time> str(curryr-10) + "-" + str(currqtr*3))
    #     yd_min = np.min(df[con].vacpercentptchg)
    #     yd_max = np.max(df[con].vacpercentptchg)
    #     con = (df.time> str(curryr) + "-" + str(currqtr*3)) & (df.vacpercentptchg > yd_max )
    #     df.loc[con, 'vacpercent'] = np.nan
    #     df['vacpercent'] = df['vacpercent'].interpolate()
    #     con = (df.time> str(curryr) + "-" + str(currqtr*3)) & (df.vacpercentptchg < yd_min )
    #     df.loc[con, 'vacpercent'] = np.nan
    #     df['vacpercent'] = df['vacpercent'].interpolate()


    #     # Recalculate vacpercentptchg
    #     df['vacpercentptchg'] = df.vacpercent.diff()


        # Forecast Rent Model (Hendershott 2002)
        ## Prepare data
        df['qfet_ln_diff'] = np.log(df['qfet']).diff()
        df['qfet_ln_lag'] = np.log(df['qfet']).shift()
        df['qfypq_ln_diff'] = np.log(df['qfypq']).diff()

        df['inv_ln_diff'] = np.log(df['inventory']).diff()
        df['inv_ln_lag'] = np.log(df['inventory']).shift()
        df['occ_ln_lag'] = np.log(1- df.vacpercent).shift()
        df['occ_ln_diff'] = np.log(1- df.vacpercent).diff()
        df['marketrentdollars_ln_lag'] = np.log(df['marketrentdollars']).shift()
        df['marketrentpctchange_lag'] = (df.marketrentpctchange).shift()

        df['USGDP_ln_diff'] = df['USGDP'].pct_change(periods=4)
        df['USGDP_ln_diff_lag'] = df['USGDP'].pct_change(periods=4).shift()
        df['gdp_ln_diff'] = df['gdp'].pct_change(periods=4)
        df['gdp_ln_diff_lag'] = df['gdp'].pct_change(periods=4).shift()
        df[['USGDP_ln_diff', 'USGDP_ln_diff_lag']] = df[['USGDP_ln_diff', 'USGDP_ln_diff_lag']].fillna(method='bfill')
        

        yd_min = np.min(df.marketrentpctchange)
        yd_max = np.max(df.marketrentpctchange)
        yd_std = np.std(df.marketrentpctchange)
        y_min = np.min(df.marketrentpctchange)
        y_max = np.max(df.marketrentpctchange)
        y_std = np.std(df.marketrentpctchange)
        y_last = list(df[["marketrentpctchange"]].dropna().marketrentpctchange)[-1]
        y_mean = np.mean(df[df.time> str(curryr-15) + "-" + str(currqtr*3)].marketrentpctchange)
        rent_last = list(df[["marketrentdollars"]].dropna().marketrentdollars)[-1]
        if yd_std > 0.01:
            y_mean = np.nanpercentile(df.marketrentpctchange, 50)
            yd_max = np.nanpercentile(df.marketrentpctchange, 65)
            yd_min = np.nanpercentile(df.marketrentpctchange, 10)
    #     print(yd_min, yd_max, yd_std, y_mean)

        ## Asking Rent
        ## Select Variables
        X_REIS = ['inv_ln_diff', 'occ_ln_diff', 'marketrentpctchange_lag']
        X_ECCA = ['qfet_ln_diff','qfet_ln_lag', 'USGDP_ln_diff', 'USGDP_ln_diff_lag']
        X_select = X_REIS + X_ECCA
        df[X_ECCA] = df[X_ECCA].fillna(method='ffill')

        X_list = X_select
        if float(df_submarket_yr[(df_submarket_yr.metrocode==metro) & (df_submarket_yr.subid== subid)].submarket_yr_begin) <= 2011:
            clf = dill.load( open('./Model_off/rent_' + metro+'_'+str(int(subid)) +'.pkl', "rb" ) )
        else:
            clf = dill.load( open('./Model_off/metro_model/rent_' + metro+ '.pkl', "rb" ) )
            adjustment_factor = df.marketrentpctchange.mean() / clf.predict(df[(df.marketrentpctchange.notna()) & (df.yr >2015) & (df.marketrentpctchange_lag.notna())][X_list]).mean()


        try:
            while df[df.yr >curryr-1].marketrentpctchange.isna().sum()>0:
                if float(df_submarket_yr[(df_submarket_yr.metrocode==metro) & (df_submarket_yr.subid== subid)].submarket_yr_begin) <= 2011:
                    df.loc[(df.marketrentpctchange.isna()) & (df.yr >2015) & (df.marketrentdollars_ln_lag.notna()), 'marketrentpctchange'] = (clf.predict(df[(df.marketrentpctchange.isna()) & (df.yr >2015) & (df.marketrentdollars_ln_lag.notna())][X_list]))
                else:
                    df.loc[(df.marketrentpctchange.isna()) & (df.yr >2015) & (df.marketrentdollars_ln_lag.notna()), 'marketrentpctchange'] = (clf.predict(df[(df.marketrentpctchange.isna()) & (df.yr >2015) & (df.marketrentdollars_ln_lag.notna())][X_list])) * adjustment_factor
                ## Adding movements
                con = (df.time> str(curryr) + "-" + str(currqtr*3)) & (df.marketrentpctchange.round(decimals= 2).diff()==0) & (df.marketrentpctchange.round(decimals= 2).diff(periods=2)==0)
                df.loc[con, "marketrentpctchange"] = df[con]["marketrentpctchange"] + (df[con]["rn"]-1)/30 * y_std
                QA_condition = (df.marketrentpctchange < yd_min )
                df.loc[QA_condition, "marketrentpctchange"] = yd_min 
                QA_condition = (df.time> str(curryr) + "-" + str(currqtr*3)) & (df.marketrentpctchange < 0) & (df.gdp.diff() > 0)
                df.loc[QA_condition, "marketrentpctchange"] = 0.001
                QA_condition = (df.time> str(curryr+5) + "-" + str(currqtr*3)) & (df.marketrentpctchange < 0.001 )
                df.loc[QA_condition, "marketrentpctchange"] = 0.001
                QA_condition = (df.marketrentpctchange > yd_max)
                df.loc[QA_condition, "marketrentpctchange"] = yd_max


                df['marketrentdollars_lag'] = df['marketrentdollars'].shift()
                df.loc[df.marketrentdollars.isna(),'marketrentdollars'] = df[df.marketrentdollars.isna()]['marketrentdollars_lag'] * (1+df[df.marketrentdollars.isna()].marketrentpctchange)
                df['marketrentdollars_ln_lag'] = np.log(df['marketrentdollars']).shift()
                df['marketrentpctchange_lag'] = (df.marketrentpctchange).shift()

        except:
            print(metro, subid, "Something in the asking rent forecast went wrong")

    #     # Limit the asking rent growth long term forecast to long term mean after 4th year
    #     LR_start = 4
    #     df['ct'] = 1
    #     df.loc[df.time< str(curryr+ LR_start) + "-" + str(currqtr*3), 'ct'] = 0
    #     ct_count = df.ct.sum()
    #     df['ct'] = df['ct'].cumsum() / ct_count


    #     y_f_last = list(df.marketrentpctchange)[-1]
    #     y_last5 = float(df[df.time== str(curryr+ LR_start) + "-" + str(currqtr*3)].marketrentpctchange)

    # #     if abs(y_f_last - y_mean) > 0  :
    # #     ratio_adj = ((y_mean + np.sign(y_f_last - y_mean))-y_last)/(y_f_last- y_last)
    #     if (abs(y_f_last - y_mean) > 0) & (abs(y_f_last- y_last5) > 0.5* yd_std):
    #         ratio_adj = ((y_mean )-y_last5)/(y_f_last- y_last5)
    #         con = (df.time> str(curryr+ LR_start) + "-" + str(currqtr*3))
    #         df.loc[con, "marketrentpctchange"] =float(y_last5) + float(ratio_adj) * (df[con].marketrentpctchange - float(y_last5))
    #     elif (abs(y_f_last - y_mean) > 0) & (abs(y_f_last- y_last5) <= 0.5* yd_std):
    #         con = (df.time> str(curryr+ LR_start) + "-" + str(currqtr*3))
    #         df.loc[con, "marketrentpctchange"] = df[con].marketrentpctchange - (y_f_last - y_mean) * df[con].ct


    #     # Recalculate asking rent
#     df.loc[df.time> str(curryr) + "-" + str(currqtr*3),'marketrentdollars'] = rent_last * (1+ df[df.time> str(curryr) + "-" + str(currqtr*3)].marketrentpctchange).cumprod() 
    
    
        ## Effective Rent (GAP)
        df['qfet_ln_diff'] = np.log(df['qfet']).diff()
        df['qfet_ln_lag'] = np.log(df['qfet']).shift()
        df['qfypq_ln_diff'] = np.log(df['qfypq']).diff()

        df['inv_ln_diff'] = np.log(df['inventory']).diff()
        df['inv_ln_lag'] = np.log(df['inventory']).shift()
        df['occ_ln_lag'] = np.log(1- df.vacpercent).shift()
        df['occ_ln_diff'] = np.log(1- df.vacpercent).diff()

        df['gap'] = (df['marketrentdollars'] - df['effrentdollars'])/df['marketrentdollars']
        df['gap_lag'] = df['gap'].shift()
        df['gap_lag_4'] = df['gap'].shift(4)
        
        ## Select Variables
        X_REIS = ['gap_lag','inv_ln_diff', 'occ_ln_diff', 'gap_lag_4']
        X_ECCA = ['qfet_ln_diff','qfet_ln_lag']
        X_select = X_REIS + X_ECCA

        X_list = X_select
        if float(df_submarket_yr[(df_submarket_yr.metrocode==metro) & (df_submarket_yr.subid== subid)].submarket_yr_begin) <= 2011:
            clf = dill.load( open('./Model_off/gap_' + metro+'_'+str(int(subid)) +'.pkl', "rb" ) )
        else:
            clf = dill.load( open('./Model_off/metro_model/gap_' + metro+ '.pkl', "rb" ) )
            adjustment_factor = df.gap.mean() / clf.predict(df[(df.gap.notna()) & (df.yr >2015) & (df.gap_lag.notna()) &  (df.gap_lag_4.notna())][X_list]).mean()


    #     y_min = df.gap.min()
        con = (df.time> str(curryr-10) + "-" + str(currqtr*3)) & (df.qtr == 4)
        df['gap_chg'] = df.gap.pct_change(periods=4)
        yd_min = np.min(df[con].gap_chg)
        yd_max = np.max(df[con].gap_chg)
        yd_std = np.std(df[con].gap)
        y_min = np.min(df[con].gap)
        y_max = np.max(df[con].gap)
        y_std = np.std(df[con].gap)
        y_last = list(df[["gap"]].dropna().gap)[-1]
        y_mean = np.mean(df[df.time> str(curryr-15) + "-" + str(currqtr*3)].gap)
    # print(yd_min, yd_max, yd_std, y_min, y_max, y_std, y_last, y_mean)

        try:
            while df[df.yr >curryr-1].gap.isna().sum()>0:
                if float(df_submarket_yr[(df_submarket_yr.metrocode==metro) & (df_submarket_yr.subid== subid)].submarket_yr_begin) <= 2011:
                    df.loc[(df.gap.isna()) & (df.yr >2015) & (df.gap_lag.notna()), 'gap'] = (clf.predict(df[(df.gap.isna()) & (df.yr >2015) & (df.gap_lag.notna())][X_list]))
                else:
                    df.loc[(df.gap.isna()) & (df.yr >2015) & (df.gap_lag.notna()), 'gap'] = (clf.predict(df[(df.gap.isna()) & (df.yr >2015) & (df.gap_lag.notna())][X_list])) *(adjustment_factor)
    #             ## Adding movements
    #             con = (df.time> str(curryr) + "-" + str(currqtr*3)) & (df.gap.round(decimals= 2).diff()==0) & (df.gap.round(decimals= 2).diff(periods=2)==0)
    #             df.loc[con, "gap"] = df[con]["gap"] + (df[con]["rn"]-1)/2 * y_std

                ## Limit gap to historical min/max
                if df_list.iloc[row].Exist == 'both':
                    QA_condition = (df.gap < y_min - 0.25* y_std)
                    df.loc[QA_condition, "gap"] = y_min - 0.25* y_std
                    QA_condition = (df.gap > y_max + 0.25*y_std)
                    df.loc[QA_condition, "gap"] = y_max+ 0.25*y_std
                else:
                    QA_condition = (df.gap < y_min)
                    df.loc[QA_condition, "gap"] = y_min - (df[QA_condition]["rn"]-1)/2 * y_std
                    QA_condition = (df.gap > y_max)
                    df.loc[QA_condition, "gap"] = y_max +  (df[QA_condition]["rn"]-1)/2 * y_std
                df.loc[(df.gap < y_min), "gap"] = y_min
                df.loc[ (df.gap > y_max), "gap"] = y_max


                # gap is non-negative
                df.loc[df.gap<0, 'gap'] = 0.

                df['gap_lag'] = df['gap'].shift()
                df['gap_lag_4'] = df['gap'].shift(4)

        except:
            print(metro, subid, "Something in the gap forecast went wrong")

        # calculate gap change
        df['gap_chg'] = df.gap.pct_change(periods=4)
        df['gap_lag_4'] = df['gap'].shift(periods=4)

        # Smooth if there is vacpercentptchg above historical min/max
        con = (df.time> str(curryr) + "-" + str(currqtr*3)) & (df.gap_chg > yd_max )
        df.loc[con, "gap"] = df[con]['gap_lag_4'] * (1+ yd_max)
        con = (df.time> str(curryr) + "-" + str(currqtr*3)) & (df.gap_chg < yd_min )
        df.loc[con, "gap"] = df[con]['gap_lag_4'] * (1+ yd_min)   



        df.loc[df.effrentdollars.isna(),'effrentdollars'] = (1- df[df.effrentdollars.isna()]['gap']) * df[df.effrentdollars.isna()]['marketrentdollars'] 

        df = pd.concat([df, df_qtr5[df_qtr5.year == yr_earlist-1]])
        df.loc[df.yr.isna(),'yr'] = df[df.yr.isna()]['year']
        df= df.sort_values(['yr', 'qtr'])
    #     df.loc[df.effrentpchange.isna(),'effrentpchange'] = df[df.effrentpchange.isna()]['effrentdollars'].pct_change(fill_method=None)
    #     df.loc[df.effrentpchange.isna(),'marketrentpctchange'] = df[df.marketrentpctchange.isna()]['marketrentdollars'].pct_change(fill_method=None)
        df['effrentpchange'] = df['effrentdollars'].pct_change(fill_method=None)
        df['marketrentpctchange'] = df['marketrentdollars'].pct_change(fill_method=None)
    
    else:
        df['effrentdollars'] = np.nan
        df['effrentpchange'] = np.nan
        df['gap'] = np.nan
        pass
   
    
#     ## Bring long term vac to ROL
#     df[['sector', 'metrocode', 'subid']] = df[['sector',  'metrocode', 'subid']].fillna(method='ffill')
#     df['subid']=df.subid.astype(int)
#     col_df_rol = ['metcode','subid', 'yr', 'rolsvac']
#     df = df.merge(df_rol[df_rol.yr>curryr][col_df_rol], on = ['metcode','subid', 'yr'], how = "left" )
#     df[['rolsvac']] = df[['rolsvac']].fillna(method='ffill')
#     con = (df.time> str(curryr+3) + "-" + str(currqtr*3)) 
#     df.loc[con, "vacpercent"] = 0.5 * df[con]["vacpercent"] + 0.5* df[con]["rolsvac"]
#     con = (df.time> str(curryr+5) + "-" + str(currqtr*3))
#     df.loc[con, "vacpercent"] = 0.5 * df[con]["vacpercent"] + 0.5* df[con]["rolsvac"]
#     con = (df.time> str(curryr+8) + "-" + str(currqtr*3))
#     df.loc[con, "vacpercent"] = 0.5 * df[con]["vacpercent"] + 0.5* df[con]["rolsvac"]
#     con = (df.time> str(curryr+9) + "-" + str(currqtr*3))
#     df.loc[con, "vacpercent"] = df[con]["rolsvac"]
    
    
#     # Recalculate vacpercentptchg
#     df['vacpercentptchg'] = df.vacpercent.diff()
    
    
    # Format output
    df[['sector', 'metrocode', 'subid']] = df[['sector',  'metrocode', 'subid']].fillna(method='ffill')
    df[['sector', 'metrocode', 'subid']] = df[['sector',  'metrocode', 'subid']].fillna(method='bfill')
    df['t'] = df.E + df.H
    df.loc[df.demolitions.isna(), 'demolitions'] = 0.
    df.loc[df.conversions.isna(),'conversions'] = 0.
    df['subid']=df.subid.astype(int)
    df.loc[df.occupiedstock.isna(), 'occupiedstock'] = round(df[df.occupiedstock.isna()].inventory * (1- df[df.occupiedstock.isna()].vacpercent))
    df['netabsorption'] = df.occupiedstock.diff()
#     df['completions'] = round(df['completions'])
#     df['inventory'] = round(df['inventory'])
#     df['marketrentdollars'] = round(df['marketrentdollars'])
#     df['effrentdollars'] = round(df['effrentdollars'])
#     df['vacpercent'] = df['vacpercent'].round(decimals= 3)
#     df['marketrentpctchange'] = df['marketrentpctchange'].round(decimals= 4)
#     df['effrentpchange'] = df['effrentpchange'].round(decimals= 7)
#     df['gap'] = df['gap'].round(decimals= 4)
    
    df = df.rename(columns = { 'inventory' : 'inv', 'completions' : 'cons', 
                                'H' : 'h', 'E' : 'e', 'demolitions' : 'demo', 'conversions' : 'conv', 
                                'occupiedstock' : 'occ', 'netabsorption' : 'abs', 
                                'marketrentdollars' : 'mrent', 'marketrentpctchange' : 'G_mrent',
                                'effrentdollars' : 'merent', 'effrentpchange' : 'G_merent',
                                'vacpercent' : 'vac'})
    
    df_qtr5 = df_qtr5.rename(columns = { 'metrocode': 'metcode', 'year': "yr", "quarter": "qtr",
                                'inventory' : 'inv', 'completions' : 'cons', 
                                'demolitions' : 'demo', 'conversions' : 'conv', 
                                'occupiedstock' : 'occ', 'netabsorption' : 'abs', 
                                'marketrentdollars' : 'mrent', 
                                'effrentdollars' : 'merent', 
                                'vacpercent' : 'vac'})
    
     
    
    df['cons_oob'] = df['cons']
    df['vac_oob'] = df['vac']
    df['abs_oob'] = df['abs']
    df['G_mrent_oob'] = df['G_mrent']
    df['G_merent_oob'] = df['G_merent']
    df['gap_oob'] = df['gap']    
    col_export = ['metcode','subid', 'yr', 'qtr', 'inv', 'cons', 'h', 'e','t', 'demo', 'conv', 'occ', 'abs', 'vac',
                 'mrent', 'G_mrent', 'merent', 'G_merent', 'gap','cons_oob', 'vac_oob' ,'abs_oob','G_mrent_oob',
                 'gap_oob']
    df = df[df.yr >= yr_earlist]
    
    try: 
        df_output
    except NameError: 
        df_output = pd.DataFrame()

    df_output = pd.concat([df_output,  df[col_export]])
    
    
    try: 
        df_output_y
    except NameError: 
        df_output_y = pd.DataFrame()
    
    col_export_y = ['metcode','subid', 'yr', 'qtr', 'inv', 'cons', 'demo', 'conv', 'occ', 'abs', 'vac',
                 'mrent',  'merent']
    df_output_y = pd.concat([df_output_y,  df_qtr5[col_export_y]])
    

# df_output.to_csv('./output_off/df_output_test.csv', index=False)
# df_output_y.to_csv('./output_off/df_output_y_test.csv', index=False)
    
    
# Aggregate to get metro level variables (Bxxx)
list_agg = ['h','e', 't', 'demo', 'conv', 'inv', 'cons', 'abs']
df_output_met = pd.DataFrame(df_output.groupby(['metcode', 'yr', 'qtr'])[list_agg].sum().reset_index())
df_output_met = df_output_met.rename(columns = {  'h' : 'Bh', 'e' : 'Be', 't' : 'Bt', 'demo' : 'Bdemo', 
                                'conv' : 'Bconv', 'inv' : 'Binv', 'cons' : 'Bcon', 'abs': 'Babs'})
temp = df_output.groupby(['metcode', 'yr', 'qtr']).apply(lambda x: pd.Series(np.average(x[["vac", "mrent", 'merent']], weights=x["inv"], axis=0), ["vac", "mrent", 'merent'])).reset_index()
temp = temp.rename(columns = {  'vac' : 'Bvac', 'mrent' : 'Bmrent', 'merent' : 'Bmerent'})
temp['Bgap'] = (temp.Bmrent - temp.Bmerent)/ temp.Bmrent
temp['G_Bmrent'] = (temp.groupby('metcode')['Bmrent']
                                  .apply(pd.Series.pct_change))
temp['G_Bmerent'] = (temp.groupby('metcode')['Bmerent']
                                  .apply(pd.Series.pct_change))
df_output_met = df_output_met.merge(temp, on = ['metcode', 'yr', 'qtr'], how = 'left' )
df_output = df_output.merge(df_output_met, on = ['metcode', 'yr', 'qtr'],how = 'left')

# Aggregate to get metro level variables for historical annual data (Bxxx)
list_agg = ['demo', 'conv', 'inv', 'cons', 'abs']
df_output_y_met = pd.DataFrame(df_output_y.groupby(['metcode', 'yr', 'qtr'])[list_agg].sum().reset_index())
df_output_y_met = df_output_y_met.rename(columns = {  'h' : 'Bh', 'e' : 'Be', 't' : 'Bt', 'demo' : 'Bdemo', 
                                'conv' : 'Bconv', 'inv' : 'Binv', 'cons' : 'Bcon', 'abs': 'Babs'})
temp = df_output_y.groupby(['metcode', 'yr', 'qtr']).apply(lambda x: pd.Series(np.average(x[["vac", "mrent", 'merent']], weights=x["inv"], axis=0), ["vac", "mrent", 'merent'])).reset_index()
temp = temp.rename(columns = {  'vac' : 'Bvac', 'mrent' : 'Bmrent', 'merent' : 'Bmerent'})
temp['Bgap'] = (temp.Bmrent - temp.Bmerent)/ temp.Bmrent
# temp['G_Bmrent'] = (temp.groupby('metcode')['Bmrent']
#                                   .apply(pd.Series.pct_change))
# temp['G_Bmerent'] = (temp.groupby('metcode')['Bmerent']
#                                   .apply(pd.Series.pct_change))
df_output_y_met = df_output_y_met.merge(temp, on = ['metcode', 'yr', 'qtr'], how = 'left' )
df_output_y = df_output_y.merge(df_output_y_met, on = ['metcode', 'yr', 'qtr'],how = 'left')


# Add annual data rows (qtr =5)
df_output_y_forecast = df_output[(df_output.qtr==4) ]
df_output_y_forecast.qtr = 5
list_agg = ['cons','h','e', 't', 'demo', 'conv', 'Bcon','Bh','Be', 'Bt', 'Bdemo', 'Bconv']
# list_agg = ['cons','h','e', 't', 'demo', 'conv']
temp = pd.DataFrame(df_output.groupby(['metcode','subid', 'yr'])[list_agg].sum().reset_index())
df_output_y_forecast = df_output_y_forecast.drop(columns=list_agg)
df_output_y_forecast = df_output_y_forecast.merge(temp, on = ['metcode','subid', 'yr'], how = "left" )


yr_connect = 2006
df_output_y = pd.concat([df_output_y[df_output_y.yr < yr_connect], 
                         df_output_y_forecast[df_output_y_forecast.yr >= yr_connect]]).reset_index(drop=True)
df_output_y= df_output_y.sort_values(['metcode', 'subid', 'yr', 'qtr'])
df_output_y['abs'] = df_output_y.groupby(['metcode', 'subid'])['occ'].apply(pd.Series.diff)
df_output_y['G_mrent'] = df_output_y.groupby(['metcode', 'subid'])['mrent'].pct_change()
df_output_y['G_merent'] = df_output_y.groupby(['metcode', 'subid'])['merent'].pct_change()
df_output_y['G_Bmrent'] = df_output_y.groupby(['metcode', 'subid'])['Bmrent'].pct_change()
df_output_y['G_Bmerent'] = df_output_y.groupby(['metcode', 'subid'])['Bmerent'].pct_change()
df_output_y.loc[df_output_y.gap.isna(), 'gap'] = (df_output_y[df_output_y.gap.isna()].mrent - df_output_y[df_output_y.gap.isna()].merent)/ df_output_y[df_output_y.gap.isna()].mrent
df_output_y.loc[df_output_y.Bgap.isna(), 'Bgap'] = (df_output_y[df_output_y.Bgap.isna()].Bmrent - df_output_y[df_output_y.Bgap.isna()].Bmerent)/ df_output_y[df_output_y.Bgap.isna()].Bmrent

df_output_y = df_output_y.drop(columns=['Babs'])
list_agg = ['abs']
df_output_y_met = pd.DataFrame(df_output_y.groupby(['metcode', 'yr'])[list_agg].sum().reset_index())
df_output_y_met = df_output_y_met.rename(columns = { 'abs': 'Babs'})
df_output_y = df_output_y.merge(df_output_y_met, on = ['metcode', 'yr'], how = 'left' )

# # # # df_output_y = df_output_y.drop(columns=['cons', 'demo', 'conv'])
# # # df_output_y = df_output_y.merge(temp.drop(columns=['cons', 'demo', 'conv']), on = ['metcode','subid', 'yr'], how = "left" )

# # # # Aggregate to get metro level variables for annual numbers(Bxxx)
# # # list_agg = ['abs']
# # # df_output_y_met = pd.DataFrame(df_output.groupby(['metcode', 'yr'])[list_agg].sum().reset_index())
# # # df_output_y_met = df_output_y_met.rename(columns = { 'abs': 'Babs'})

# # # temp = df_output_y.groupby(['metcode', 'yr']).apply(lambda x: pd.Series(np.average(x[["vac", "mrent", 'merent']], weights=x["inv"], axis=0), ["vac", "mrent", 'merent'])).reset_index()
# # # temp = temp.rename(columns = {  'vac' : 'Bvac', 'mrent' : 'Bmrent', 'merent' : 'Bmerent'})
# # # temp['Bgap'] = (temp.Bmrent - temp.Bmerent)/ temp.Bmrent
# # # temp['G_Bmrent'] = (temp.groupby('metcode')['Bmrent']
# # #                                   .apply(pd.Series.pct_change))
# # # temp['G_Bmerent'] = (temp.groupby('metcode')['Bmerent']
# # #                                   .apply(pd.Series.pct_change))
# # # df_output_y = df_output_y.merge(temp, on = ['metcode', 'yr'], how = 'left' )
# # # df_output_y = df_output_y.merge(df_output_y_met, on = ['metcode', 'yr'], how = 'left' )


# # df_output_y = pd.concat([df_output_y[df_output_y.yr<curryr],  df_output_y_forecast[df_output_y_forecast.yr>=curryr]])
df_output_y['cons_oob'] = df_output_y['cons']
df_output_y['vac_oob'] = df_output_y['vac']
df_output_y['abs_oob'] = df_output_y['abs']
df_output_y['G_mrent_oob'] = df_output_y['G_mrent']
df_output_y['G_merent_oob'] = df_output_y['G_merent']
df_output_y['gap_oob'] = df_output_y['gap'] 





df_output = pd.concat([df_output,  df_output_y])
df_output = df_output.sort_values(['metcode', 'subid', 'yr', 'qtr'])
df_output = df_output.reset_index()

col_df_rol = ['metcode','subid', 'yr', 'qtr', 'rol_h', 'rol_e', 'rolsvac', 'rolscon', 'Brolabs',
              'rolcons', 'rolsabs', 'rolvac', 'rolsmre', 'rolmrent', 'grolsmre',
                'rolmere', 'grolsmer', 'Brolmeren', 'rolsgap', 'Brolgap']
# print(df_output.columns)
# temp = df_output.copy()
# print(df_rol.columns)

df_output = df_output.merge(df_rol[col_df_rol], on = ['metcode','subid', 'yr', 'qtr'], how = "left" )

df_output['temp'] = df_output.groupby(['metcode', 'subid'])['mrent'].pct_change()
df_output.loc[df_output.G_mrent.isna(),'G_mrent'] = df_output[df_output.G_mrent.isna()]['temp']
df_output['temp'] = df_output.groupby(['metcode', 'subid'])['merent'].pct_change()
df_output.loc[df_output.G_merent.isna(),'G_merent'] = df_output[df_output.G_merent.isna()]['temp']
df_output['temp'] = df_output.groupby(['metcode', 'subid'])['Bmrent'].pct_change()
df_output.loc[df_output.G_Bmrent.isna(),'G_Bmrent'] = df_output[df_output.G_Bmrent.isna()]['temp']
df_output['temp'] = df_output.groupby(['metcode', 'subid'])['Bmerent'].pct_change()
df_output.loc[df_output.G_Bmerent.isna(),'G_Bmerent'] = df_output[df_output.G_Bmerent.isna()]['temp']



col_export= [ 'metcode', "subid", "yr", "qtr", "inv", "cons", "rolscon", "h", "rol_h",
             "e", "t", "Bcon", "rolcons", "demo", "conv", "occ", "abs", "rolsabs",
             "Babs", "Brolabs", "vac", "rolsvac", "Bvac", "rolvac", "mrent", "rolsmre",
             "Bmrent", "rolmrent", "G_mrent", "grolsmre", "G_Bmrent",  "merent", "rolmere",
             "G_merent", "grolsmer", "Bmerent","Brolmeren", "G_Bmerent" ,
             "gap", "rolsgap", "Bgap", "Brolgap",'cons_oob', 'vac_oob' ,'abs_oob','G_mrent_oob',
             'G_merent_oob',    'gap_oob']
df_output = df_output[col_export]

condition = (df_output.G_mrent.isna()) & (df_output.grolsmre.notna())
df_output.loc[condition, "G_mrent"] = df_output[condition].grolsmre
condition = (df_output.G_merent.isna()) & (df_output.grolsmer.notna())
df_output.loc[condition, "G_merent"] = df_output[condition].grolsmer

df_output['cons_oob'] = df_output['cons']
df_output['vac_oob'] = df_output['vac']
df_output['abs_oob'] = df_output['abs']
df_output['G_mrent_oob'] = df_output['G_mrent']
df_output['G_merent_oob'] = df_output['G_merent']
df_output['gap_oob'] = df_output['gap'] 



df_output.to_csv('./output_off/aptsubdiag_' + str(curryr) + 'q' + str(currqtr) + '_OOB_v1.csv', index=False)
df_output.to_csv('/home/central/subcast/data/off/forecast/current/test2022/offsubdiag_' + str(curryr) + 'q' + str(currqtr) + '_OOB.csv', index=False)

end_time = time.time()
print("Done! Quarterly Forecast in " + sector + " sector takes ", (end_time-start_time)/60, " minutes." ) 
print("Forecast ends at ",time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time)))


Forecast starts at  2022-07-14 20:42:20
There are 846 submarkets in the submarket trend and  846 submarkets in the aptsubtrnds in  Off sector.
There are 846 submarkets in the aptsubtrnds after excluding subid 90


Office Submarket Forecast Progress:   0%|          | 0/846 [00:00<?, ?it/s]

Done! Quarterly Forecast in Off sector takes  16.37383950551351  minutes.
Forecast ends at  2022-07-14 20:58:43


In [78]:
# df[df.yr<2022][['metrocode', 'subid', 'yr', 'qtr','Year', 'Qtr', 'year', 'quarter','completions', 'E','H', 'threshold', 'inventory'] + X_select]
# df[df.yr>=2022][['metrocode', 'subid', 'yr', 'qtr','Year', 'Qtr', 'year', 'quarter','gap'] + X_select]
# print(metro, subid)
df[(df.vacpercentptchg.notna()) & (df.yr >2015) & (df.vacpercentptchg.notna())][X_list]

,gap_lag,inv_ln_diff,occ_ln_diff,gap_lag_4,qfet_ln_diff,qfet_ln_lag
1,0.153976,0.000000,0.004219,NaN,0.004525,7.113329
2,0.153331,0.085816,0.009429,NaN,0.007429,7.117854
3,0.148627,0.000000,0.002083,NaN,0.004014,7.125283
4,0.147106,0.000000,0.000000,0.153976,0.005275,7.129298
5,0.147106,0.000000,-0.002083,0.153331,-0.104688,7.134572
6,0.147882,0.000000,-0.002088,0.148627,0.051547,7.029884
7,0.150319,0.000000,-0.003140,0.147106,0.024790,7.081431
8,0.155138,0.000000,0.000000,0.147106,0.006514,7.106221
9,0.152955,0.000000,0.002094,0.147882,0.008897,7.112735
10,0.151019,0.000000,-0.001047,0.150319,0.008330,7.121632


In [14]:
df_rol[df_rol.yr==2021]

,metcode,subid,yr,qtr,rolscon,rol_h,rol_e,rolcons,rolsabs,Brolabs,rolsvac,rolvac,rolsmre,rolmrent,grolsmre,rolmere,grolsmer,Brolmeren,rolsgap,Brolgap
107,AQ,1,2021,1,0.0,0.0,0.0,0,-2.0,-30,0.047000,0.033712,828.229980,923.411804,0.003976,790.270020,0.003174,893.687805,0.045833,0.032189
108,AQ,1,2021,2,92.0,92.0,0.0,227,51.0,131,0.055000,0.035821,853.429993,952.123718,0.030426,810.840027,0.026029,921.221619,0.049904,0.032456
109,AQ,1,2021,3,0.0,0.0,0.0,0,34.0,348,0.048000,0.027538,947.429993,1081.968994,0.110144,904.599976,0.115633,1052.667969,0.045207,0.027081
110,AQ,1,2021,5,92.0,92.0,0.0,227,86.0,444,0.047010,0.027781,956.229980,1089.000000,0.159134,902.729980,0.145926,1057.000000,0.055951,0.029385
227,AQ,2,2021,1,0.0,0.0,0.0,0,-7.0,-30,0.034000,0.033712,1012.489990,923.411804,-0.006983,983.950012,-0.007565,893.687805,0.028188,0.032189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108463,WS,2,2021,5,905.0,905.0,387.0,1598,1063.0,1610,0.059000,0.071990,2952.080078,2540.000000,0.070544,2839.620117,0.077136,2444.000000,0.038094,0.037795
108555,WS,3,2021,1,0.0,0.0,0.0,175,2.0,189,0.043000,0.074244,1892.469971,2293.647949,0.003995,1828.930054,0.003726,2202.962891,0.033575,0.039537
108556,WS,3,2021,2,0.0,0.0,0.0,188,0.0,155,0.043000,0.074554,1886.770020,2302.931885,-0.003012,1823.369995,-0.003040,2213.870117,0.033602,0.038673
108557,WS,3,2021,3,0.0,0.0,0.0,0,0.0,250,0.043000,0.069297,2023.209961,2527.898926,0.072314,1955.229980,0.072317,2431.668945,0.033600,0.038067


In [60]:
df[df.completions.notnull()].completions.mean() 

11538.461538461539

In [76]:
# clf.predict(df[df.completions.notnull() & (df.completions_lag.notnull()) & (df.yr >2015) & (df.inv_lag.notna())][X_list]).mean()
df[(df.gap.notna()) & (df.yr >2015) & (df.gap_lag.notna())][X_list]

,gap_lag,inv_ln_diff,occ_ln_diff,gap_lag_4,qfet_ln_diff,qfet_ln_lag
1,0.153976,0.000000,0.004219,NaN,0.004525,7.113329
2,0.153331,0.085816,0.009429,NaN,0.007429,7.117854
3,0.148627,0.000000,0.002083,NaN,0.004014,7.125283
4,0.147106,0.000000,0.000000,0.153976,0.005275,7.129298
5,0.147024,0.000000,-0.002083,0.153331,-0.104688,7.134572
6,0.147882,0.000000,-0.002088,0.148627,0.051547,7.029884
7,0.150319,0.000000,-0.003140,0.147106,0.024790,7.081431
8,0.155138,0.000000,0.000000,0.147024,0.006514,7.106221
9,0.152955,0.000000,0.002094,0.147882,0.008897,7.112735
10,0.151019,0.000000,-0.001047,0.150319,0.008330,7.121632
